# Project, Part 1, Attempt 1
# Rohan Krishnamurthi
# 9/5/2024

# 0: initial preparation of folders, files

In [1]:
# Load Packages
import pybedtools
import os
from pybedtools import BedTool
import pandas as pd

In [2]:
# Set working directory
os.chdir('/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project')

In [3]:
os.environ['PATH'] += ':/opt/homebrew/bin/bedtools'

In [4]:
print(os.environ['PATH'])

/opt/homebrew/bin:/opt/homebrew/sbin:/Users/rohankrishnamurthi/anaconda3/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/TeX/texbin:/Applications/quarto/bin:/opt/homebrew/bin/bedtools


In [53]:
#cd("/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research")

In [5]:
# Read in file with region of 10,000 BP from transcription start site
kBP_10_df = pd.read_table('TF_TSS_10kb_named.bed', header = None)
kBP_10_df[4] = (kBP_10_df[1] + kBP_10_df[2])/2

# Convert data frame to bedtool
kBP_10_updated = BedTool.from_dataframe(kBP_10_df)

In [6]:
# Set list of cell lines and histone marks
lines = ["A549", "H1", "HepG2", "K562", "GM12878", "MCF-7"]
marks = ["H3K4me3", "H3K27Ac", "H3K9me3", "H3K27me3"]

In [7]:
# Get list of folders (for loop 1)
folders = []
for i in lines:
        for j in marks:
                folder = f"/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/{i}/{j}"
                folders.append(folder)

In [9]:
#folders

In [8]:
# Define function to get list of files from a folder
def list_files(folder):
    files = []
    for entry in os.listdir(folder):
        if os.path.isfile(os.path.join(folder, entry)):
            files.append(entry)
    return files

In [9]:
# Get dictionary of folder and relevant files (for loop 2)
dictionary = {}
for i in lines:
        for j in marks:
                folder = f"/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/{i}/{j}/"
                files = [file for file in list_files(folder) if (file != ".DS_Store" and file != ".bed") ]
                dictionary[folder] = files

In [10]:
dictionary

{'/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/A549/H3K4me3/': ['ENCFF001WUM.bed.gz',
  'ENCFF001WUL.bed.gz'],
 '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/A549/H3K27Ac/': [],
 '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/A549/H3K9me3/': ['ENCFF407FDL.bed.gz',
  'ENCFF155GOT.bed.gz',
  'ENCFF911EGN.bed.gz'],
 '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/A549/H3K27me3/': [],
 '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/H1/H3K4me3/': ['ENCFF001SVC.bed.gz'],
 '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/H1/H3K27Ac/': ['ENCFF001SUX.bed.gz'],
 '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Pro

# 1: Histone Data

## 1a: individual histone marks

In [11]:
x = pybedtools.BedTool()
word_to_remove = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/"
save_folder = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_processed_data"

for folder in dictionary:
        name = folder
        name_abb = ((folder.replace(word_to_remove, "")).replace("/","_"))[:-1]
        files = dictionary[folder]
        
        if len(files) >= 1:
            bed_path = f'{save_folder}/{name_abb}.bed.gz'
            csv_path = f'{save_folder}/{name_abb}_consensus_all.csv'
            bed_path_2 = f'{save_folder}/{name_abb}_consensus_all.bed'
        
            intersection_list =[]
            
            for file in files: 
                intersection_list.append(BedTool(name + file).fn)
            
            intersection_list.append(kBP_10_updated.fn)
            
            result = x.multi_intersect(i=intersection_list)
            result.saveas(bed_path)
            
            len_files = len(files)
            n = len_files+5
            column_mapping = {
                0: 'chrom',
                1: 'start',
                2: 'end',
                3: "peak_count",
                4: "files",
                n: "TF_TSS_10kb_named.bed"
            }

            for i in range(0, len_files):
                file = files[i]
                column_mapping[i+5] = f"{file}"


            df = pd.read_csv(bed_path, sep='\t', header=None)
            df = df.rename(columns=column_mapping)
            
            # Ensure peaks are found in all replicate experiments
            df_filtered = df[df["peak_count"] == max(df["peak_count"])]
            
            #df_filtered.to_csv(csv_path, index = False)
            df_filtered.to_csv(bed_path_2, sep='\t', index=False, header=False)
            


/var/folders/p9/mp8rwqxx7kj78jky4wxl4x_m0000gn/T/ipykernel_96977/1318227349.py:41: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(bed_path, sep='\t', header=None)
/var/folders/p9/mp8rwqxx7kj78jky4wxl4x_m0000gn/T/ipykernel_96977/1318227349.py:41: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(bed_path, sep='\t', header=None)


## 1b: bivalency, H3K4me3 + H3K27me3

In [ ]:
# A549: none
# H1: 2 files

In [12]:
keys = list(dictionary.keys())
values = list(dictionary.values())

In [14]:
keys_updated = []
for key in keys:
    if ("H3K4me3" in key or "H3K27me3" in key):
        keys_updated.append(key)


In [15]:
keys_A549 = []
for key in keys:
    if ("H3K4me3" in key or "H3K27me3" in key):
        if "A549" in key:
            keys_A549.append(key)

keys_H1 = []
for key in keys:
    if ("H3K4me3" in key or "H3K27me3" in key):
        if "H1" in key:
            keys_H1.append(key)
            
keys_HepG2 = []
for key in keys:
    if ("H3K4me3" in key or "H3K27me3" in key):
        if "HepG2" in key:
            keys_HepG2.append(key)
            
keys_K562 = []
for key in keys:
    if ("H3K4me3" in key or "H3K27me3" in key):
        if "K562" in key:
            keys_K562.append(key)
            
keys_GM1287 = []
for key in keys:
    if ("H3K4me3" in key or "H3K27me3" in key):
        if "GM1287" in key:
            keys_GM1287.append(key)
            
keys_MCF_7 = []
for key in keys:
    if ("H3K4me3" in key or "H3K27me3" in key):
        if "MCF-7" in key:
            keys_MCF_7.append(key)

In [16]:
dict_A549 = {k: dictionary[k] for k in keys_A549}
dict_H1 = {k: dictionary[k] for k in keys_H1}
dict_HepG2 = {k: dictionary[k] for k in keys_HepG2}
dict_K562 = {k: dictionary[k] for k in keys_K562}
dict_GM1287 = {k: dictionary[k] for k in keys_GM1287}
dict_MCF_7 = {k: dictionary[k] for k in keys_MCF_7}


In [17]:
#dict_A549

{'/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/A549/H3K4me3/': ['ENCFF001WUM.bed.gz',
  'ENCFF001WUL.bed.gz'],
 '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/A549/H3K27me3/': []}

In [88]:
#A549: no files

#H1: one file, one file

#HepG2: three files, three files    
    
#K562: three files, one file

#GM1287: 4, 3

#MCF-7: two files, no file

In [64]:
#pwd('/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/')

In [18]:
#H1: one file, one file
x = pybedtools.BedTool()
word_to_remove = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/"
save_folder = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_processed_data"

H1_list = ['/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/H1/H3K4me3/ENCFF001SVC.bed.gz', 
           '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/H1/H3K27me3/ENCFF001SUY.bed.gz']

H1_list_short = ['H1/H3K4me3/ENCFF001SVC.bed.gz', 'H1/H3K27me3/ENCFF001SUY.bed.gz']

bed_path = f'{save_folder}/H1_bivalency.bed.gz'
csv_path = f'{save_folder}/H1_bivalency_consensus_all.csv'
bed_path_2 = f'{save_folder}/H1_bivalency_consensus_all.bed'

intersection_list = []
for file in H1_list:
    intersection_list.append(BedTool(file).fn)
intersection_list.append(kBP_10_updated.fn)  
    
result = x.multi_intersect(i=intersection_list)
result.saveas(bed_path)

len_files = len(H1_list_short)
n = len_files+5
column_mapping = {
    0: 'chrom',
    1: 'start',
    2: 'end',
    3: "peak_count",
    4: "files",
    n: "TF_TSS_10kb_named.bed"
}

for i in range(0, len_files):
    file = H1_list_short[i]
    column_mapping[i+5] = f"{file}"


df = pd.read_csv(bed_path, sep='\t', header=None)
df = df.rename(columns=column_mapping)

# Ensure peaks are found in all replicate experiments
df_filtered = df[df["peak_count"] == max(df["peak_count"])]

df_filtered.to_csv(csv_path, index = False)
df_filtered.to_csv(bed_path_2, sep='\t', index=False, header=False)

In [19]:
#HepG2: three files, three files
x = pybedtools.BedTool()
word_to_remove = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/"
save_folder = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_processed_data"

HepG2_list = ['/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/HepG2/H3K4me3/ENCFF001XDC.bed.gz',
              '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/HepG2/H3K4me3/ENCFF001XDB.bed.gz',
              '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/HepG2/H3K4me3/ENCFF001SWO.bed.gz',
              '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/HepG2/H3K27me3/ENCFF001SWL.bed.gz',
              '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/HepG2/H3K27me3/ENCFF001XCT.bed.gz',
              '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/HepG2/H3K27me3/ENCFF001XCW.bed.gz',]

HepG2_list_short = ['HepG2/H3K4me3/ENCFF001XDC.bed.gz',
              'HepG2/H3K4me3/ENCFF001XDB.bed.gz',
              'HepG2/H3K4me3/ENCFF001SWO.bed.gz',
              'HepG2/H3K27me3/ENCFF001SWL.bed.gz',
              'HepG2/H3K27me3/ENCFF001XCT.bed.gz',
              'HepG2/H3K27me3/ENCFF001XCW.bed.gz',]

bed_path = f'{save_folder}/HepG2_bivalency.bed.gz'
csv_path = f'{save_folder}/HepG2_bivalency_consensus_all.csv'
bed_path_2 = f'{save_folder}/HepG2_bivalency_consensus_all.bed'

intersection_list = []
for file in HepG2_list:
    intersection_list.append(BedTool(file).fn)
intersection_list.append(kBP_10_updated.fn)  
    
result = x.multi_intersect(i=intersection_list)
result.saveas(bed_path)

len_files = len(HepG2_list_short)
n = len_files+5
column_mapping = {
    0: 'chrom',
    1: 'start',
    2: 'end',
    3: "peak_count",
    4: "files",
    n: "TF_TSS_10kb_named.bed"
}

for i in range(0, len_files):
    file = HepG2_list_short[i]
    column_mapping[i+5] = f"{file}"


df = pd.read_csv(bed_path, sep='\t', header=None)
df = df.rename(columns=column_mapping)

# Ensure peaks are found in all replicate experiments
df_filtered = df[df["peak_count"] == max(df["peak_count"])]

df_filtered.to_csv(csv_path, index = False)
df_filtered.to_csv(bed_path_2, sep='\t', index=False, header=False)

In [20]:
#K562: three files, one file
x = pybedtools.BedTool()
word_to_remove = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/"
save_folder = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_processed_data"

K562_list = ['/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/K562/H3K4me3/ENCFF001XGU.bed.gz',
             '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/K562/H3K4me3/ENCFF001SZJ.bed.gz',
             '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/K562/H3K4me3/ENCFF001XGT.bed.gz',
             '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/K562/H3K27me3/ENCFF001SZF.bed.gz']

K562_list_short = ['K562/H3K4me3/ENCFF001XGU.bed.gz',
             'K562/H3K4me3/ENCFF001SZJ.bed.gz',
             'K562/H3K4me3/ENCFF001XGT.bed.gz',
             'K562/H3K27me3/ENCFF001SZF.bed.gz']

bed_path = f'{save_folder}/K562_bivalency.bed.gz'
csv_path = f'{save_folder}/K562_bivalency_consensus_all.csv'
bed_path_2 = f'{save_folder}/K562_bivalency_consensus_all.bed'

intersection_list = []
for file in K562_list:
    intersection_list.append(BedTool(file).fn)
intersection_list.append(kBP_10_updated.fn)  
    
result = x.multi_intersect(i=intersection_list)
result.saveas(bed_path)

len_files = len(K562_list_short)
n = len_files+5
column_mapping = {
    0: 'chrom',
    1: 'start',
    2: 'end',
    3: "peak_count",
    4: "files",
    n: "TF_TSS_10kb_named.bed"
}

for i in range(0, len_files):
    file = K562_list_short[i]
    column_mapping[i+5] = f"{file}"


df = pd.read_csv(bed_path, sep='\t', header=None)
df = df.rename(columns=column_mapping)

# Ensure peaks are found in all replicate experiments
df_filtered = df[df["peak_count"] == max(df["peak_count"])]

df_filtered.to_csv(csv_path, index = False)

df_filtered.to_csv(bed_path_2, sep='\t', index=False, header=False)


In [21]:
#GM1287: 4, 3
x = pybedtools.BedTool()
word_to_remove = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/"
save_folder = "/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_processed_data"

GM1287_list = ['/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/GM12878/H3K4me3/ENCFF001WYJ.bed.gz',
               '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/GM12878/H3K4me3/ENCFF001SUF.bed.gz',
               '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/GM12878/H3K4me3/ENCFF001WYK.bed.gz',
               '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/GM12878/H3K4me3/ENCFF001SUM.bed.gz',
               '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/GM12878/H3K27me3/ENCFF001WYB.bed.gz',
              '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/GM12878/H3K27me3/ENCFF001SUI.bed.gz',
              '/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project/histone_data/GM12878/H3K27me3/ENCFF001WYE.bed.gz']

GM1287_list_short = ['GM12878/H3K4me3/ENCFF001WYJ.bed.gz',
  'GM12878/H3K4me3/ENCFF001SUF.bed.gz',
  'GM12878/H3K4me3/ENCFF001WYK.bed.gz',
  'GM12878/H3K4me3/ENCFF001SUM.bed.gz',
 'GM12878/H3K4me3/ENCFF001WYB.bed.gz', 
 'GM12878/H3K4me3/ENCFF001SUI.bed.gz', 
 'GM12878/H3K4me3/ENCFF001WYE.bed.gz']


bed_path = f'{save_folder}/GM12878_bivalency.bed.gz'
csv_path = f'{save_folder}/GM12878_bivalency_consensus_all.csv'
bed_path = f'{save_folder}/GM12878_bivalency_consensus_all.bed'

intersection_list = []
for file in GM1287_list:
    intersection_list.append(BedTool(file).fn)
intersection_list.append(kBP_10_updated.fn)  
    
result = x.multi_intersect(i=intersection_list)
result.saveas(bed_path)

len_files = len(GM1287_list_short)
n = len_files+5
column_mapping = {
    0: 'chrom',
    1: 'start',
    2: 'end',
    3: "peak_count",
    4: "files",
    n: "TF_TSS_10kb_named.bed"
}

for i in range(0, len_files):
    file = GM1287_list_short[i]
    column_mapping[i+5] = f"{file}"


df = pd.read_csv(bed_path, sep='\t', header=None)
df = df.rename(columns=column_mapping)

# Ensure peaks are found in all replicate experiments
df_filtered = df[df["peak_count"] == max(df["peak_count"])]

df_filtered.to_csv(csv_path, index = False)
df_filtered.to_csv(bed_path_2, sep='\t', index=False, header=False)


